In [13]:
import pandas as pd
import numpy as np
import time


dataset = pd.read_csv(r'hackathon_basketball_distribution.csv')

shotTypes = dataset.complexShotType.unique()
shotIds = [i for i in range(len(shotTypes))]

dataset['complexShotType'].replace(shotTypes, shotIds, inplace=True)

In [17]:
df1 = dataset.sample(frac=0.5, random_state=int(time.time()))
df2 = dataset.drop(df1.index)
dataset.head()


player  season  fga3  fg3  period  startGameClock  shotClock  \
0    Murillo, Josue    2019     1    0       4           48.08       7.60   
1  Castaneda, Urian    2019     1    1       4           37.42       1.80   
2     Dean, Gregory    2021     1    1       2          297.60       8.36   
3    Suazo, Garrett    2020     1    0       2          313.00      16.56   
4   Hatcher, Damian    2019     1    0       2          243.24      19.08   

               region   locX   locY  ...  complexShotType       contestLevel  \
0    right wing three -24.80 -18.64  ...                0  lightly_contested   
1  right corner three -44.74 -22.17  ...                0  lightly_contested   
2        middle three -17.76   9.28  ...                1  lightly_contested   
3    right wing three -26.89 -20.08  ...                1        uncontested   
4    right wing three -31.96 -22.33  ...                1        uncontested   

   closestDefDist  rearViewContest releaseTime  shotAngle  shooterSpeed  \
0            8.91                0        0.64  47.718662      0.131762   
1            8.34                0        0.84  97.680960      2.189780   
2            6.96                0        1.20 -21.147858      1.133456   
3           10.73                0        0.88  53.497086      2.296208   
4           11.32                0        1.28  66.326220      1.166667   

   shooterVelAngle  rimDepth  rimLeftRight  
0       -66.153610  1.066764     -0.674009  
1        13.513096  0.364760      0.096667  
2        57.175232  1.681004     -0.661988  
3       -41.983253 -0.663653     -0.857141  
4        23.673782 -0.270286      0.182177  

[5 rows x 23 columns]

In [18]:
feature_cols = ['startGameClock', 'shotClock', 'distance', 'dribblesBefore', 'assistOpp', 'complexShotType', 'closestDefDist', 'rearViewContest', 'shotAngle', 'shooterSpeed', 'shooterVelAngle']
x1 = df1.loc[:, feature_cols]
x2 = df2.loc[:, feature_cols]

y1 = df1['fg3']
y2 = df2['fg3']

In [19]:
from sklearn import tree

clf1 = tree.DecisionTreeClassifier()
clf2 = tree.DecisionTreeClassifier()

# 3. fit 
clf1 = clf1.fit(x1, y1)

clf2 = clf2.fit(x2, y2)

In [8]:
prediction = clf.predict(testing_data.loc[:, feature_cols])
print(prediction)
accuracy = np.count_nonzero(prediction == testing_data['fg3'])/prediction.size
print(accuracy)

[0 1 1 ... 0 1 1]
0.5464765802037269


In [ ]:
players = dataset.player.unique()
df = pd.DataFrame()
THRESHOLD = 50 # minimum predicted number of shots to be considered
df['player_name'] = []
df['predicted_number_made'] = []
df['actual_number_made'] = []
df['percent_difference'] = []
df['fg%'] = []
for player in players:
    player_shots = testing_data[testing_data['player'] == player]
    if(len(player_shots) > THRESHOLD):
        player_predicted = clf.predict(player_shots.loc[:, feature_cols])
        player_actual = player_shots['fg3']
        predicted_total = np.sum(player_predicted)
        actual_total = np.sum(player_actual)
        df = df.append({'player_name' : player, 'predicted_number_made' : 
                        predicted_total, 'actual_number_made' : actual_total, 
                        'percent_difference': (actual_total - predicted_total)/predicted_total, 
                        'fg%' : actual_total/len(player_shots) }, ignore_index = True)